In [3]:
import pickle

import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures

In [4]:
df = pd.read_csv('../data/insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [5]:
X = df.drop(columns=['charges'])
y = df['charges']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=11)

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print("#" * 30)
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (936, 6)
y_train shape: (936,)
##############################
X_test shape: (402, 6)
y_test shape: (402,)


In [6]:
num_vars = ['age', 'bmi', 'children']
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy='mean')),
           ("scaler", StandardScaler())]
)

cat_vars = ['sex', 'smoker', 'region']
categorical_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy='most_frequent')),
           ("ohe", OneHotEncoder(handle_unknown="ignore", drop='first'))])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_vars),
        ("cat", categorical_transformer, cat_vars)
    ]
)
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['age', 'bmi', 'children']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ohe',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'))]),
                                 ['sex', 'smoker', 'region'])])

In [7]:
polyreg = make_pipeline(PolynomialFeatures(degree=3, include_bias=True), LinearRegression())

In [8]:
pipe = Pipeline([('preprocessor', preprocessor),
                 ('polyreg', polyreg)])

pipe

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'bmi', 'children']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['sex', 'smoker',
                                                   'region'])])),
                ('polyreg',
                 Pipeline(steps=[('polynomialfeatures',
                                  PolynomialFeatures(degree=3)),
                                 ('linearregression', LinearRegression())]))])

In [9]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'bmi', 'children']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['sex', 'smoker',
                                                   'region'])])),
                ('polyreg',
                 Pipeline(steps=[('polynomialfeatures',
                                  PolynomialFeatures(degree=3)),
                                 ('linearregression', LinearRegression())]))])

In [10]:
poly = pipe.fit(X_train, y_train)
poly

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'bmi', 'children']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['sex', 'smoker',
                                                   'region'])])),
                ('polyreg',
                 Pipeline(steps=[('polynomialfeatures',
                                  PolynomialFeatures(degree=3)),
                                 ('linearregression', LinearRegression())]))])

In [11]:
poly_predict_train = poly.predict(X_train)

In [12]:
poly_predict_test = poly.predict(X_test)

In [13]:
r2_score_train = poly.score(X_train, y_train)
mse_train = mean_squared_error(poly_predict_train, y_train)
mae_train = mean_absolute_error(poly_predict_train, y_train)

r2_score_test = r2_score(poly_predict_test, y_test)
mae = mean_absolute_error(poly_predict_test, y_test)
mse = mean_squared_error(poly_predict_test, y_test)

In [14]:
print(f"R2 Score is: {r2_score_train}")
print(f"Mean Squared error: {mse_train}")
print(f"Mean Absolute Error: {mae_train} ")

R2 Score is: 0.8645131946390517
Mean Squared error: 20211677.77521373
Mean Absolute Error: 2759.4889405787385 


In [15]:
print(f"R2 Score is: {r2_score_test}")
print(f"Mean Squared error: {mse}")
print(f"Mean Absolute Error: {mae} ")

R2 Score is: 0.7961253191952915
Mean Squared error: 25833076.506456424
Mean Absolute Error: 3192.2981918905375 


In [17]:
with open(r"../models/model.pickle", "wb") as model:
    pickle.dump(pipe, model)